In [1]:
# Setup iniziale con i vari import da sfruttare per i due svolgimenti separati
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [2]:
# Caricamento Dataset di Kaggle
# Kaggle "DOM_hourly": columns => ["Datetime", "DOM_MW"]
df = pd.read_csv("DOM_hourly.csv", parse_dates=["Datetime"])
# Forziamo che la colonna 'Datetime' sia in formato datetime
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.set_index('Datetime', inplace=True)

In [3]:
# Feature engineering
df["hour"] = df.index.hour
df["dayofweek"] = df.index.dayofweek
df["month"] = df.index.month

In [5]:
# Calcolo delle varie medie, settimanale, mensile e globale
media_settimanale = df['DOM_MW'].resample('W').mean()
media_mensile = df['DOM_MW'].resample('ME').mean()
media_globale = df['DOM_MW'].mean()

# Aggiungere le medie al DataFrame
df['Media_Settimanale'] = media_settimanale.reindex(df.index).ffill()
df['Media_Mensile'] = media_mensile.reindex(df.index).ffill()
df['Media_Globale'] = media_globale

### SVOLGIMENTO RAPIDO USANDO DIRETTAMENTE PANDAS

In [6]:
# Classifica il consumo in base alla media settimanale, mensile e globale
def classificazione(consumo, media_settimanale, media_mensile, media_globale):
    if consumo > media_settimanale:
        return 'alto consumo (settimanale)'
    elif consumo > media_mensile:
        return 'alto consumo (mensile)'
    elif consumo > media_globale:
        return 'alto consumo (globale)'
    else:
        return 'basso consumo'

df['Classificazione_Pandas'] = df.apply(lambda row: classificazione(row['DOM_MW'], row['Media_Settimanale'], row['Media_Mensile'], row['Media_Globale']), axis=1)

# Visualizza il risultato
print(df[['DOM_MW', 'Media_Settimanale', 'Media_Mensile', 'Media_Globale', 'Classificazione_Pandas']])

                      DOM_MW  Media_Settimanale  Media_Mensile  Media_Globale  \
Datetime                                                                        
2005-12-31 01:00:00   9389.0                NaN            NaN   10949.203625   
2005-12-31 02:00:00   9070.0                NaN            NaN   10949.203625   
2005-12-31 03:00:00   9001.0                NaN            NaN   10949.203625   
2005-12-31 04:00:00   9042.0                NaN            NaN   10949.203625   
2005-12-31 05:00:00   9132.0                NaN            NaN   10949.203625   
...                      ...                ...            ...            ...   
2018-01-01 20:00:00  18418.0          17920.625   13873.676075   10949.203625   
2018-01-01 21:00:00  18567.0          17920.625   13873.676075   10949.203625   
2018-01-01 22:00:00  18307.0          17920.625   13873.676075   10949.203625   
2018-01-01 23:00:00  17814.0          17920.625   13873.676075   10949.203625   
2018-01-02 00:00:00  17428.0

### SVOLGIMENTO CON MODELLO DI MACHINE LEARNING

In [8]:
# Target binario: 1 se consumo sopra la mediana
df["target_median"] = (df["DOM_MW"] > df["DOM_MW"].median()).astype(int)

# #### Selezione delle feature
X = df[["hour", "dayofweek", "month"]]
y = df["target_median"]

# #### Split del dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.25, random_state=42
)

# #### Modello 1 – Decision Tree
tree = DecisionTreeClassifier(max_depth=5, random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

print("Decision Tree:")
print(classification_report(y_test, y_pred_tree, digits=3))

# #### Modello 2 – Neural Network (MLP)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)
mlp.fit(X_train_scaled, y_train)
y_pred_mlp = mlp.predict(X_test_scaled)

print("Neural Network:")
print(classification_report(y_test, y_pred_mlp, digits=3))

Decision Tree:
              precision    recall  f1-score   support

           0      0.758     0.839     0.796     14524
           1      0.820     0.732     0.773     14524

    accuracy                          0.785     29048
   macro avg      0.789     0.785     0.785     29048
weighted avg      0.789     0.785     0.785     29048

Neural Network:
              precision    recall  f1-score   support

           0      0.841     0.753     0.794     14524
           1      0.776     0.857     0.815     14524

    accuracy                          0.805     29048
   macro avg      0.809     0.805     0.805     29048
weighted avg      0.809     0.805     0.805     29048

